In [1]:
import re
import pandas as pd

In [2]:
with open('raw_tex_big.txt', 'r') as f:
    raw_tex = f.readlines()

In [3]:
raw_tex

['\\begin{table*}[!htbp]\n',
 '\\centering\n',
 '%\\resizebox{0.87\\linewidth}{!}{\n',
 '\\caption{The classification accuracy (\\%) over the methods on six large-scale datasets released in~\\cite{lim2021large}.\n',
 'The error bar ($\\pm$) denotes the standard deviation score of results over 10 trials. \n',
 'We highlight the best score on each dataset in bold and the runner-up score with underline.\n',
 'Note that OOM refers to the out-of-memory error.}\n',
 '\\label{tab:result_large}\n',
 '\\resizebox{0.8\\linewidth}{!}\n',
 '{\n',
 '\\begin{tabular}{|c|c|c|c|c|c|c|c|}\n',
 '\\hline\n',
 '                    & \\textbf{Penn94} & \\textbf{pokec}     & \\textbf{arXiv-year} & \\textbf{snap-patents} & \\textbf{genius} & \\textbf{twitch-gamers} &  \\parbox[t]{2mm}{\\multirow{5}{*}{\\rotatebox[origin=c]{90}{\\textbf{Avg. Rank}}}} \\\\\n',
 '\\textbf{ Edge Hom.~\\cite{zhu2020beyond}}  &  0.47     &    0.44       &  0.22  & 0.07   & 0.61  &  0.54 &  \\\\\n',
 '\\textbf{\\#Nodes}  &   41,554

In [4]:
raw_tex[18].split('&')

['     MLP$^\\dagger$                    ',
 '  $ 73.61 \\pm 0.40 $  ',
 '     $ 62.37 \\pm 0.02$        ',
 '  $ 36.70 \\pm 0.21 $  ',
 '  $ 31.34 \\pm 0.05 $  ',
 '  $ 86.68 \\pm 0.09 $ ',
 ' $ 60.92 \\pm 0.07 $  ',
 ' 9.67\\\\\n']

In [5]:
raw_tex[30].split('&')

['      \\ada++               ',
 '   $  \\bm{85.74  \\pm 0.42} $   ',
 '  $  \\bm{83.05\\pm 0.07}$       ',
 ' $ 54.79  \\pm 0.25$ ',
 ' $  \\underline{62.03 \\pm 0.21} $ ',
 ' $ \\bm{90.91 \\pm 0.13}$ ',
 ' $ \\bm{66.34 \\pm 0.29}$  ',
 '  1.33   \\\\  \\hline\n']

In [6]:
raw_tex[30].split('&')[0].strip()

'\\ada++'

In [7]:
re.findall(r'\d+\.*\d*', raw_tex[30].split('&')[1])

['85.74', '0.42']

In [8]:
datasets = ['penn94', 'pokec', 'arxiv-year', 'snap-patents', 'genius', 'twitch-gamers']
length = len(datasets)

In [9]:
model_name_list = []
result_list = [[] for _ in range(length)]

for i in range(18, 31):
    text_list =  raw_tex[i].split('&')
    if not text_list[0].strip().startswith('%'):
        model_name_list.append(text_list[0].strip())
        for i in range(1, length+1):
            res = re.findall(r'\d+\.*\d*', text_list[i])
            if len(res) == 2:
                result_list[i-1].append(float(res[0]))
            else:
                result_list[i-1].append(0)

In [10]:
model_name_list

['MLP$^\\dagger$',
 'GCN$^\\dagger$',
 'GAT$^\\dagger$',
 'MixHop$^\\dagger$',
 'GCN\\rom{2}$^\\dagger$',
 'H$2$GCN',
 'GPR-GNN$^\\dagger$',
 'GGCN',
 'ACM-GCN',
 'LINKX$^\\dagger$',
 '\\ada',
 '\\ada++']

In [11]:
result_list

[[73.61,
  82.47,
  81.53,
  83.47,
  82.92,
  81.31,
  81.38,
  0,
  82.52,
  84.71,
  85.57,
  85.74],
 [62.37, 75.45, 71.77, 81.07, 78.94, 0, 78.83, 0, 63.81, 82.04, 83.0, 83.05],
 [36.7,
  46.02,
  46.05,
  51.81,
  47.21,
  49.09,
  45.07,
  0,
  47.37,
  56.0,
  54.68,
  54.79],
 [31.34, 45.65, 45.37, 52.16, 37.88, 0, 40.19, 0, 55.14, 61.95, 62.09, 62.03],
 [86.68, 87.42, 55.8, 90.58, 90.24, 0, 90.05, 0, 80.33, 90.77, 90.66, 90.91],
 [60.92, 62.18, 59.89, 65.64, 63.39, 0, 61.89, 0, 62.01, 66.06, 66.19, 66.34]]

In [12]:
result_dict = {
    'model': model_name_list,
}

for i in range(length):
    result_dict[datasets[i]] = result_list[i]

In [13]:
df = pd.DataFrame.from_dict(result_dict)
df

,model,penn94,pokec,arxiv-year,snap-patents,genius,twitch-gamers
0,MLP$^\dagger$,73.61,62.37,36.70,31.34,86.68,60.92
1,GCN$^\dagger$,82.47,75.45,46.02,45.65,87.42,62.18
2,GAT$^\dagger$,81.53,71.77,46.05,45.37,55.80,59.89
3,MixHop$^\dagger$,83.47,81.07,51.81,52.16,90.58,65.64
4,GCN\rom{2}$^\dagger$,82.92,78.94,47.21,37.88,90.24,63.39
5,H$2$GCN,81.31,0.00,49.09,0.00,0.00,0.00
6,GPR-GNN$^\dagger$,81.38,78.83,45.07,40.19,90.05,61.89
7,GGCN,0.00,0.00,0.00,0.00,0.00,0.00
8,ACM-GCN,82.52,63.81,47.37,55.14,80.33,62.01
9,LINKX$^\dagger$,84.71,82.04,56.00,61.95,90.77,66.06


In [14]:
datasets = df.columns.tolist()[1: ]
datasets

['penn94', 'pokec', 'arxiv-year', 'snap-patents', 'genius', 'twitch-gamers']

In [15]:
for i in range(len(datasets)):
    column_name = datasets[i] + '_rank'
    df[column_name] = df[datasets[i]].rank(ascending=False)

In [16]:
df['average_rank'] = 0
for i in range(len(datasets)):
    column_name = datasets[i] + '_rank'
    df['average_rank'] += df[column_name]

df['average_rank'] /= length

In [17]:
df

,model,penn94,pokec,arxiv-year,snap-patents,genius,twitch-gamers,penn94_rank,pokec_rank,arxiv-year_rank,snap-patents_rank,genius_rank,twitch-gamers_rank,average_rank
0,MLP$^\dagger$,73.61,62.37,36.70,31.34,86.68,60.92,11.0,10.0,11.0,10.0,8.0,9.0,9.833333
1,GCN$^\dagger$,82.47,75.45,46.02,45.65,87.42,62.18,7.0,7.0,9.0,6.0,7.0,6.0,7.000000
2,GAT$^\dagger$,81.53,71.77,46.05,45.37,55.80,59.89,8.0,8.0,8.0,7.0,10.0,10.0,8.500000
3,MixHop$^\dagger$,83.47,81.07,51.81,52.16,90.58,65.64,4.0,4.0,4.0,5.0,4.0,4.0,4.166667
4,GCN\rom{2}$^\dagger$,82.92,78.94,47.21,37.88,90.24,63.39,5.0,5.0,7.0,9.0,5.0,5.0,6.000000
5,H$2$GCN,81.31,0.00,49.09,0.00,0.00,0.00,10.0,11.5,5.0,11.5,11.5,11.5,10.166667
6,GPR-GNN$^\dagger$,81.38,78.83,45.07,40.19,90.05,61.89,9.0,6.0,10.0,8.0,6.0,8.0,7.833333
7,GGCN,0.00,0.00,0.00,0.00,0.00,0.00,12.0,11.5,12.0,11.5,11.5,11.5,11.666667
8,ACM-GCN,82.52,63.81,47.37,55.14,80.33,62.01,6.0,9.0,6.0,4.0,9.0,7.0,6.833333
9,LINKX$^\dagger$,84.71,82.04,56.00,61.95,90.77,66.06,3.0,3.0,1.0,3.0,2.0,3.0,2.500000


In [18]:
df.to_excel('results_big_rank.xls', header=True, index=False, encoding='utf-8')